# 自定义层
>我们可以用创造性的方式组合不同的层,从而设计出适用于各种任务的框架.
>例如,研究人员发明了专门用于处理图像,文本,序列数据和执行动态规划的层,有时候我们会遇到或者要自己发明一个现在在深度学习框架中还不存在的层,在这些情况下必须构建自定义层

In [1]:
#不带参数的层
import torch
import torch.nn.functional as F
from torch import nn


class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self,X):
        return X-X.mean()

#提供一些数据,验证是否能按照预期工作
layer=CenteredLayer()
layer(torch.FloatTensor([1,2,3,4,5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [2]:
#将层作为组件合并到更复杂的模型中
net=nn.Sequential(nn.Linear(8,128),CenteredLayer())

Y=net(torch.rand(4,8))
Y.mean()

tensor(4.6566e-09, grad_fn=<MeanBackward0>)

In [5]:
#带参数的层
class MyLinear(nn.Module):
    def __init__(self,in_units,units):
        super().__init__()
        self.weight=nn.Parameter(torch.randn(in_units,units))
        self.bias=nn.Parameter(torch.randn(units,))

    def forward(self,X):
        linear=torch.matmul(X,self.weight.data)+self.bias.data
        return F.relu(linear)
    
#实例化类并访问其模型参数
linear=MyLinear(5,3)
linear.weight

Parameter containing:
tensor([[ 0.2837, -0.7344,  0.5914],
        [-0.9803, -0.2031,  1.3133],
        [-0.7906,  0.0293,  0.9432],
        [-2.1069, -1.6502,  0.5487],
        [-0.4713,  0.9797, -0.9952]], requires_grad=True)

In [13]:
#使用自定义层执行前向传播计算
linear(torch.rand(2,5))

#使用自定义层构建模型,就像使用内置的全连接层一样使用自定义层
net=nn.Sequential(MyLinear(64,8),MyLinear(8,1))
net(torch.rand(2,64))

tensor([[4.5666],
        [3.6774]])